# Oligopoly Simulation Demo

This notebook demonstrates the oligopoly simulation platform for market competition modeling.

## What is Oligopoly?

A comprehensive platform for simulating oligopoly market competition with:

1. **Market Competition**: Cournot & Bertrand models with learning strategies
2. **Policy Analysis**: Taxes, subsidies, price caps with event logging
3. **Interactive Visualization**: Profit surface heatmaps and market dynamics
4. **REST API**: FastAPI endpoints for simulation and analysis

**Key Features:**
- Multi-firm competition with Nash equilibrium computation
- Policy intervention modeling and impact analysis
- Learning strategies and collusion dynamics
- Real-time dashboard and data export capabilities

## Setup and Installation

In [ ]:
! [ ! -d "oligopoly" ] && git clone https://github.com/bangyen/oligopoly.git
! cd oligopoly && pip install -e .

print("Setup complete!")

## Imports and Configuration

In [ ]:
import os

os.chdir('./oligopoly')
print(f"Current working directory: {os.getcwd()}")

In [ ]:
import logging
import warnings

import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go

# Import oligopoly simulation modules
from src.sim.heatmap.cournot_heatmap import compute_cournot_heatmap
from src.sim.strategies.nash_strategies import (
    bertrand_nash_equilibrium,
    cournot_nash_equilibrium,
)

# Suppress economic validation warnings for cleaner demo output
logging.getLogger("src.sim.validation.economic_validation").setLevel(logging.ERROR)
logging.getLogger("src.sim.games.cournot").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning)

# Set random seeds for reproducibility
np.random.seed(42)

print("Imports complete!")

## Cournot vs Bertrand Competition

In [ ]:
def quick_demo():
    print("Oligopoly Simulation Demo - Cournot vs Bertrand")
    print("=" * 50)

    # Define market parameters
    a, b = 100.0, 1.0  # P = 100 - Q
    costs = [10.0, 15.0]  # Two firms with different costs

    print(f"Market: P = {a} - {b}Q")
    print(f"Firms: Cost1={costs[0]}, Cost2={costs[1]}")

    # Calculate Nash equilibrium for Cournot competition
    print("\n--- Cournot Competition (Quantity) ---")
    cournot_quantities, cournot_price, cournot_profits = cournot_nash_equilibrium(
        a, b, costs
    )
    cournot_total_quantity = sum(cournot_quantities)
    cournot_cs = 0.5 * b * cournot_total_quantity**2  # Consumer surplus = 0.5 * b * Q^2

    print(f"Market Price: ${cournot_price:.2f}")
    print(f"Total Quantity: {cournot_total_quantity:.2f}")
    print(f"Firm Quantities: {[f'{q:.2f}' for q in cournot_quantities]}")
    print(f"Firm Profits: {[f'{p:.2f}' for p in cournot_profits]}")
    print(f"Consumer Surplus: ${cournot_cs:.2f}")

    # Calculate Nash equilibrium for Bertrand competition
    print("\n--- Bertrand Competition (Price) ---")
    bertrand_prices, bertrand_quantities, bertrand_profits, bertrand_price = (
        bertrand_nash_equilibrium(a, b, costs)
    )
    bertrand_total_quantity = sum(bertrand_quantities)
    bertrand_cs = 0.5 * b * bertrand_total_quantity**2

    print(f"Market Price: ${bertrand_price:.2f}")
    print(f"Total Quantity: {bertrand_total_quantity:.2f}")
    print(f"Firm Prices: {[f'{p:.2f}' for p in bertrand_prices]}")
    print(f"Firm Quantities: {[f'{q:.2f}' for q in bertrand_quantities]}")
    print(f"Firm Profits: {[f'{p:.2f}' for p in bertrand_profits]}")
    print(f"Consumer Surplus: ${bertrand_cs:.2f}")

    # Compare results
    print("\n--- Comparison ---")
    print(f"Price Difference: ${bertrand_price - cournot_price:.2f}")
    print(
        f"Quantity Difference: {bertrand_total_quantity - cournot_total_quantity:.2f}"
    )
    print(f"CS Difference: ${bertrand_cs - cournot_cs:.2f}")

    return (cournot_price, cournot_quantities, cournot_profits, cournot_cs), (
        bertrand_price,
        bertrand_quantities,
        bertrand_profits,
        bertrand_cs,
    )


# Run the demo
cournot_result, bertrand_result = quick_demo()

## Policy Intervention

In [ ]:
# Policy intervention demo
def policy_demo():
    print("Policy Intervention Demo - Tax Impact")
    print("=" * 40)

    # Baseline market
    a, b = 100.0, 1.0
    baseline_costs = [10.0, 15.0]

    # Run baseline Cournot
    baseline_quantities, baseline_price, baseline_profits = cournot_nash_equilibrium(
        a, b, baseline_costs
    )
    baseline_total_quantity = sum(baseline_quantities)
    baseline_cs = 0.5 * b * baseline_total_quantity**2

    print(f"Baseline - Price: ${baseline_price:.2f}, CS: ${baseline_cs:.2f}")

    # Apply 20% tax (simulated by increasing costs)
    taxed_costs = [12.0, 18.0]  # 20% cost increase
    taxed_quantities, taxed_price, taxed_profits = cournot_nash_equilibrium(
        a, b, taxed_costs
    )
    taxed_total_quantity = sum(taxed_quantities)
    taxed_cs = 0.5 * b * taxed_total_quantity**2

    print(f"With Tax - Price: ${taxed_price:.2f}, CS: ${taxed_cs:.2f}")

    # Calculate impact
    price_increase = taxed_price - baseline_price
    cs_decrease = baseline_cs - taxed_cs
    profit_change = sum(taxed_profits) - sum(baseline_profits)

    print("\nTax Impact:")
    print(f"Price Increase: ${price_increase:.2f}")
    print(f"Consumer Surplus Loss: ${cs_decrease:.2f}")
    print(f"Profit Change: ${profit_change:.2f}")
    print(f"Deadweight Loss: ${cs_decrease - profit_change:.2f}")

    return (baseline_price, baseline_quantities, baseline_profits, baseline_cs), (
        taxed_price,
        taxed_quantities,
        taxed_profits,
        taxed_cs,
    )


baseline, taxed = policy_demo()

## Profit Surface Heatmap

In [ ]:
# Generate profit surface heatmap
def create_heatmap():
    print("Generating Cournot Profit Surface Heatmap...")

    # Market parameters
    a, b = 100.0, 1.0
    costs = [10.0, 15.0]

    # Create quantity grids
    q1_range = np.linspace(0, 50, 20)
    q2_range = np.linspace(0, 50, 20)

    # Temporarily suppress all warnings during heatmap computation
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        # Also suppress logging during computation
        old_level = logging.getLogger("sim.games.cournot").level
        logging.getLogger("sim.games.cournot").setLevel(logging.CRITICAL)

        try:
            # Compute profit surface for firm 1
            profit_matrix, _, _ = compute_cournot_heatmap(
                a, b, costs, 0, 1, q1_range, q2_range, []
            )
        finally:
            # Restore logging level
            logging.getLogger("sim.games.cournot").setLevel(old_level)

    # Create heatmap
    fig = go.Figure(
        data=go.Heatmap(
            z=profit_matrix,
            x=q2_range,
            y=q1_range,
            colorscale="Viridis",
            colorbar=dict(title="Firm 1 Profit"),
        )
    )

    fig.update_layout(
        title="Cournot Competition: Firm 1 Profit Surface",
        xaxis_title="Firm 2 Quantity",
        yaxis_title="Firm 1 Quantity",
        width=600,
        height=500,
    )

    # Try to display the figure, with fallback
    try:
        fig.show()
    except Exception as e:
        print(f"Could not display interactive plot: {e}")
        print("Creating matplotlib version instead...")

        # Create matplotlib version
        plt.figure(figsize=(8, 6))
        plt.imshow(
            profit_matrix,
            extent=[q2_range[0], q2_range[-1], q1_range[0], q1_range[-1]],
            origin="lower",
            cmap="viridis",
            aspect="auto",
        )
        plt.colorbar(label="Firm 1 Profit")
        plt.xlabel("Firm 2 Quantity")
        plt.ylabel("Firm 1 Quantity")
        plt.title("Cournot Competition: Firm 1 Profit Surface")
        plt.show()

        # Also save as HTML
        fig.write_html("cournot_heatmap.html")
        print("Heatmap also saved as 'cournot_heatmap.html'")

    # Find Nash equilibrium point
    nash_q1 = (a - 2 * costs[0] + costs[1]) / (3 * b)
    nash_q2 = (a - 2 * costs[1] + costs[0]) / (3 * b)

    print(f"Nash Equilibrium: Firm 1 = {nash_q1:.2f}, Firm 2 = {nash_q2:.2f}")

    # Print some key profit values
    print(f"Max profit in grid: ${np.max(profit_matrix):.2f}")
    print(f"Min profit in grid: ${np.min(profit_matrix):.2f}")

    return fig


heatmap_fig = create_heatmap()

## Key Takeaways

This demo shows the oligopoly simulation platform capabilities:

1. **Market Competition**: Compare Cournot vs Bertrand models with different outcomes
2. **Policy Analysis**: Quantify the impact of taxes on prices and consumer welfare  
3. **Visualization**: Generate profit surface heatmaps to understand strategic interactions
4. **Economic Insights**: Analyze Nash equilibria, consumer surplus, and market dynamics

### Next Steps

- Run the interactive dashboard: `python dashboard/main.py` then visit `http://localhost:5000`
- Try the API: `uvicorn src.main:app --reload` then visit `/docs`
- Explore learning strategies: `python scripts/strategy_demo.py`
- Run batch experiments: `python experiments/cli.py experiments/sample_config.json`

For more information, visit the [GitHub repository](https://github.com/bangyen/oligopoly).
